In [1]:
import pyspark
import os

myConf=pyspark.SparkConf()
spark = pyspark.sql.SparkSession\
    .builder\
    .master("local")\
    .appName("myApp")\
    .config(conf=myConf)\
    .getOrCreate()

In [2]:
scoreList =[("김하나","English", 100),("김하나","Math", 80),
            ("임하나","English", 70),("임하나","Math", 100),
            ("김갑돌","English", 82.3),("김갑돌","Math", 98.5)]
_scoreRdd=spark.sparkContext.parallelize(scoreList)

#문제1#

In [3]:
namesum=_scoreRdd.map(lambda x: (x[0],x[2])).reduceByKey(lambda x,y:(x+y))

In [4]:
print(namesum.collect())

[('김하나', 180), ('임하나', 170), ('김갑돌', 180.8)]


#문제2#

In [7]:
subjectsum=_scoreRdd.map(lambda x: (x[1],x[2])).reduceByKey(lambda x,y:(x+y))

In [10]:
print(subjectsum.collect())

[('English', 252.3), ('Math', 278.5)]


#문제3#

In [11]:
nameCombine=_scoreRdd.map(lambda x: (x[0],x[2]))\
            .combineByKey(lambda value: (value,1),
             lambda x,value: (x[0]+value, x[1]+1),
             lambda x,y: (x[0]+y[0], x[1]+y[1]))

In [12]:
print(nameCombine.collect())

[('김하나', (180, 2)), ('임하나', (170, 2)), ('김갑돌', (180.8, 2))]


#문제4#

In [13]:
averageByKey = nameCombine.map(lambda x:(x[0],x[1][0]/x[1][1]))
averageByKey.collectAsMap()

{'김하나': 90.0, '임하나': 85.0, '김갑돌': 90.4}